## XGboost unhealthy feature selection

- Parallel implementation of feature selection specifically for AWS
- Each iteration is saved as you go along in case the kernel is interupted
- All available cores are used


In [1]:
!pip install pillow==4.1.1

     |████████████████████████████████| 5.7 MB 14.4 MB/s eta 0:00:01
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.2.0
    Uninstalling Pillow-7.2.0:
      Successfully uninstalled Pillow-7.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.16.2 requires pillow>=4.3.0, but you have pillow 4.1.1 which is incompatible.
matplotlib 3.3.4 requires pillow>=6.2.0, but you have pillow 4.1.1 which is incompatible.
bokeh 2.2.3 requires pillow>=7.1.0, but you have pillow 4.1.1 which is incompatible.


In [ ]:
!conda install -y -c conda-forge xgboost

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/linux-64::jupyter_server==1.4.1=py36h5fab9bb_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::bokeh==2.2.3=py36h5fab9bb_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/noarch::aiobotocore==1.2.1=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-spyder==0.3.2=pyhd8ed1ab_0
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.3.4=py36hd391965_0
  - conda-forge/linux-64::spyder==4.2.0=py36h5fab9bb_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::seaborn-base==0.11.1=pyhd8ed1ab_1
  - conda-forge/noarch::sphinx==3.5.1=pyhd8ed1ab_0
  - conda-

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
plt.rcdefaults()

In [ ]:
alz_blood_all = pd.read_csv('alz_blood_all.csv', index_col=0,low_memory=False)
#alz_blood_all = alz_blood_all.set_index('sample_id')
alz_blood_all

In [ ]:
#Drop columns with >10% NAs
nas=alz_blood_all.isnull().sum()
nas=nas[1:]
        
for i in nas:
    if i>=len(alz_blood_all.index)/10:
        try:
            alz_blood_all=alz_blood_all.drop(nas.keys()[i], axis=1)
        except KeyError:
            pass

In [ ]:
alz_blood_all

In [ ]:
#Function to do test-train split with mean imputation
#Index: ID, Col 0: AGE (no NAs), Col 1+: CpGs
def mean_impute(data):
    nas=data.isnull().sum()    
    col_means=data.mean(axis=0)
    na_cols=[]
    na_cols_means=[]

    for i in range(len(nas)):
        if nas[i]!=0:        
            na_cols.append(nas.keys()[i])
            na_cols_means.append(col_means[i])
        
    ids=list(data.index)
    for i in ids:
        for j in range(len(na_cols)):
            if str(data.loc[i][na_cols[j]])=="nan":
                data.loc[i][na_cols[j]]=na_cols_means[j]
    
    
    return data

In [ ]:
# Takes df of importance scores produced by repeat_XGBoost and sorts by mean imp score
def importances_sorted_by_mean(df_imp):
    df_imp_sorted = df_imp.sort_values('Mean', ascending=False)
    return df_imp_sorted

# Makes a histogram of the frequecy of cgs in the top (top_num) importance scores
def histogram_of_top_CpGs_by_importance(df_imp, top_num=20):
    vs = []
    inds = []
    for col in df_imp.columns[:-2]:
        c = df_imp[col].sort_values(ascending=False)
        vs.append(c[:top_num])
        inds  = inds + list(c.index[:top_num])
    h = pd.Series(inds).value_counts()
    # Plotting the first 100 importance scores
    plt.figure(figsize = (30, 12))
    plt.bar(h.index[:100],h[:100])
    plt.title('Frequency of CpGs in the top 100 importances')
    plt.ylabel('Frequency')
    plt.xlabel('CpG')
    plt.xticks(fontsize = 16, rotation=90)
    plt.show()
    return inds, vs, h

In [9]:
# Finding cpgs with on average the highest importance scores, see mean columnn in df below
#df_imp_sorted = importances_sorted_by_mean(df_imp)
#df_imp_sorted

In [10]:
# This function makes a histogram of the frequency of cpgs in the top 100 in the 50 trials above
#inds, vs, h = histogram_of_top_CpGs_by_importance(df_imp)

In [3]:
import multiprocessing as mp
mp.cpu_count()

16

Starting trial 10
Starting trial 11
Starting trial 12
Starting trial 13
Starting trial 14
Starting trial 15
Starting trial 16
Starting trial 17
Starting trial 18
Starting trial 19
Starting trial 20
Starting trial 21
Starting trial 22
Starting trial 23
Starting trial 24
Starting trial 25


In [ ]:
import multiprocessing as mp
import _pickle as cPickle


#Df=q
def one_fit(X,y,test_size, i):
    print('Starting trial ' + str(i))

    X_tr, X_tst, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=i)
    X_train = mean_impute(X_tr)
    X_test = mean_impute(X_tst)

    XG = XGBRegressor(objective='reg:squarederror',
                n_estimators=200,
                min_child_weight=1,
                max_depth=3,
                subsample=0.7,
                colsample_bytree=0.5,
                learning_rate=0.1)

    eval_set = [(X_train, y_train),(X_test, y_test)]
    XG.fit(X_train, y_train, eval_metric="rmse", early_stopping_rounds = 10, eval_set=eval_set, verbose=False)

    #preds_train = XG.predict(X_train)
    #preds_test = XG.predict(X_test)
    #rms_train.append((mean_squared_error(y_train, preds_train))**0.5)
    #rms_test.append((mean_squared_error(y_test, preds_test))**0.5)
    #r2_train.append(r2_score(y_train, preds_train))
    #r2_test.append(r2_score(y_test, preds_test))
    print('Ending trial ' + str(i))
    file = 'xgb_imp_alz_blood_'+str(i)
    with open(file, 'wb') as fp:
        cPickle.dump(XG.feature_importances_, fp)
        
    return XG.feature_importances_

def rep_xgb(df,iterations=50,test_size=0.25,early_stopping_rounds=10):

    ncpu=mp.cpu_count()
    X = df.iloc[:,1:]
    y = df.iloc[:,0]
    
    pool = mp.Pool(ncpu)
    results = [pool.apply_async(one_fit,    args=(X,y,test_size, i)) for i in [2,3,4,8,10,12,17,20,22,23,25,30,31,32,33,46,47,49]]
    importances = [p.get() for p in results]
    pool.close()
    pool.join()
    
    #df_imp = pd.DataFrame(importances, columns = df.columns[1:])
    #df_imp = df_imp.transpose()
    #cols = []
    #for i in range(1,iterations+1):
        #cols.append('trial_'+str(i))
    #df_imp.columns = cols
    #df_imp['Mean'] = df_imp.mean(axis=1)
    #df_imp['Std'] = df_imp.std(axis=1)
    
    return importances
    





In [12]:
#file = 'xgb_imp_park_blood_'+str(2)
#with open(r"xgb_imp_park_blood_2", "rb") as input_file:
    #trial = cPickle.load(input_file)
#trial

In [ ]:
%%time
imp_alz_blood_all=rep_xgb(alz_blood_all,iterations=50)


Starting trial 0
Starting trial 1
Starting trial 2
Starting trial 3
Starting trial 4
Starting trial 5
Starting trial 6
Starting trial 7
Starting trial 8
Starting trial 9
Starting trial 10
Starting trial 11
Starting trial 12
Starting trial 13
Starting trial 14
Starting trial 15
Starting trial 16
Starting trial 17
Starting trial 18
Starting trial 19
Starting trial 20
Starting trial 21
Starting trial 22
Starting trial 23
Starting trial 24
Starting trial 25
Starting trial 26
Starting trial 27
Starting trial 28
Starting trial 29
Starting trial 30
Starting trial 31
Starting trial 32
Starting trial 33
Starting trial 34
Starting trial 35
Ending trial 13
Starting trial 36
Starting trial 37
Ending trial 1
Starting trial 38
Ending trial 28
Starting trial 39
Ending trial 18
Starting trial 40
Ending trial 24
Starting trial 41
Ending trial 11
Starting trial 42
Ending trial 16
Starting trial 43
Ending trial 9
Starting trial 44
Starting trial 45


Process ForkPoolWorker-38:
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
MemoryError
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.

Starting trial 48
Ending trial 27
Ending trial 5
Ending trial 0
Ending trial 26
Ending trial 21
Ending trial 14
Ending trial 7
Ending trial 35
Ending trial 15
Ending trial 19
Ending trial 29
Ending trial 6
Ending trial 34


In [ ]:
file = 'xgb_imp_park_blood_all.csv'
imp_park_blood_all.to_csv(file)

In [17]:
imp_park_blood_all

,trial_1,trial_2,trial_3,trial_4,trial_5,Mean,Std
cg00050873,0.036211,0.036595,0.018020,0.047033,0.024760,0.032524,0.010113
cg00212031,0.048078,0.046578,0.059213,0.040443,0.059545,0.050771,0.007480
cg00213748,0.000000,0.000000,0.000000,0.000000,0.049529,0.009906,0.019812
cg00214611,0.045916,0.063088,0.033133,0.043157,0.056335,0.048326,0.010444
cg00455876,0.036821,0.038455,0.040642,0.041370,0.043483,0.040154,0.002315
cg01707559,0.044100,0.060216,0.058890,0.060506,0.047639,0.054270,0.006971
cg02004872,0.044332,0.038084,0.053020,0.044935,0.038704,0.043815,0.005389
cg02011394,0.048895,0.043364,0.049808,0.058596,0.048873,0.049907,0.004908
cg02050847,0.056570,0.037465,0.062141,0.049514,0.061134,0.053365,0.009114
cg02233190,0.044017,0.053667,0.093656,0.050950,0.041726,0.056803,0.018937
